In [1]:
from gensim.summarization import summarize
from newspaper import Article
from model import Model
from utils import build_dict, build_dataset, batch_iter
from rouge import Rouge
import pickle
import pandas as pd
import re
import tensorflow as tf
import random

import nltk 
from nltk.tag import pos_tag
from nltk import FreqDist
from nltk.tokenize import word_tokenize

import sym_vocab
import ant_vocab

nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
import numpy as np
import os


# download 
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaehyungseo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# 지문 생성 및 문제 생성을 위한 텍스트 파일 생성
def crawl_news(url, raw_txt_name, num_words):
    # URL 기반 크롤링, 언어는 영어로 설정
    news = Article(url, language = 'en') # URL 가져오기
    news.download() # 해당 url의 html을 가져오기
    # HTML 파싱 적용
    news.parse() 
    # 지정된 단어 갯수를 기준으로 Textrank를 적용하여 파싱된 텍스트를 요약 (Rule base)
    text = summarize(news.text, word_count = num_words) # 600자 기준으로 하기
    title = news.title # 파싱한 html에서 title에 해당하는 부분 추출하기
    title = title.lower() # title에 대해서 전부 소문자화 
    
    title = re.sub('\([^)]*\)', '', title) # 괄호를 포함한 문자열 제거 
    
    # 크롤링한 raw data를 별도로 저장
    with open(raw_txt_name, 'w', encoding = 'utf-8') as f: # 크롤링한 raw data를 저장하기
        f.write(text)
        
    # 크롤링한 뉴스 기사 문단 분리 없이 변수 할당
    text = re.sub('\n', ' ', text) # raw data에서 불필요한 줄바꿈표 제거 (상대적일듯)

    return text, title

In [3]:
# 일치 지문 생성을 위한 함수를 별도로 정의
def bf_create_correct(raw_txt_name, prepro_txt_name):
    # 문장 번호: 문장 형식의 저장을 위해 변수 생성
    test_dict = {}
    
    # 저장했었던 크롤링 본문 데이터 불러오기
    with open(raw_txt_name, 'r', encoding = 'utf-8') as f:
    # raw data 상태에서 전처리를 통해 모델에 들어갈 수 있는 데이터셋으로 변형
        for idx, text in enumerate(f.readlines()):
            # print(idx, text)
            text = text.lower() # 소문자로 통일
            text = re.sub('[.]', '', text) # 일단 온점 모두 제거
            text = re.sub('[,]', ' ,', text) # 반점 하나의 단어로 인식
            text = re.sub("\’s",' \'s', text) # 소유격도 하나의 단어로 인식
            text = re.sub("[0-9]", '#', text) # 숫자를 #로 치환
            #text = re.sub("'.+'", '', text) # 작은 따옴표 안 고유명사 제거 
            text = re.sub('[“”]', '', text) # 큰 따옴표 제거 
            text = re.sub('\n', '', text) # 줄 바꿈표 제거 
            print(idx, text)
        
            # 사전에 저장하기
            test_dict[idx] = text ## {1: '텍스트', 2: '텍스트'}
    
    # 전처리된 데이터를 별도의 파일로 저장 
    with open(prepro_txt_name, 'w', encoding = 'utf-8') as f: # 전처리한 데이터를 저장하기 
        # 줄 바꿈을 기준으로 문장을 구분하여 저장 
        for i in range(len(test_dict)):
            test_dict[i] = test_dict[i] + " " + "." + '\n'
            f.write(test_dict[i])
    
    return test_dict 

In [4]:
# 텍스트 요약 학습 데이터 불러오기 
def shake_bot_correct(filename, vaild_article_path):
    # 설정된 세팅 값 불러오기
    with open("args.pickle", "rb") as f:
        args = pickle.load(f)
        
    # 실행시 최초의 상태로 초기화하여 모델 재사용이 가능하도록 함
    tf.reset_default_graph()
    # 학습된 단어 사전과 본문 최대 길이와 요약 최대 길이 설정 값을 불러옴
    print("셰봇이 단어 사전 불러오는 중...")
    word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("valid", args.toy)
    # 불러온 설정 값에 따라서 검증 데이터셋을 생성
    print("셰봇이 기본 설정 값 초기화 중...")
    valid_x = build_dataset("valid", vaild_article_path, word_dict, article_max_len, summary_max_len, args.toy)
    valid_x_len = [len([y for y in x if y != 0]) for x in valid_x]

    with tf.Session() as sess:
        # 학습된 체크포인트와 모델을 불러오기 
        print("Loading saved model...")
        model = Model(reversed_dict, article_max_len, summary_max_len, args, forward_only=True) # 검증 단계
        saver = tf.train.Saver(tf.global_variables())
        ckpt = tf.train.get_checkpoint_state("./saved_model/")
        saver.restore(sess, ckpt.model_checkpoint_path)
        
        # 검증을 진행할 배치 사이즈 설정
        batches = batch_iter(valid_x, [0] * len(valid_x), args.batch_size, 1)
        
        # 검증 단계인 만큼 인코더 인풋 부분만 고려하여 학습 진행
        print("셰봇이 수능 일치 문제 생성 준비 중.. {}...".format(filename))
        for batch_x, _ in batches:
            # 문장 -> 문장 예측이 가능하도록 데이터 분할 
            batch_x_len = [len([y for y in x if y != 0]) for x in batch_x]
            # 검증을 위한 배치에 따라 모델 및 데이터 셋 설정 준비 
            valid_feed_dict = {
                model.batch_size: len(batch_x),
                model.X: batch_x,
                model.X_len: batch_x_len,
            }
            # Prediction을 목적으로 학습된 모델을 검증을 위한 설정 값을 적용하여 예측 결과 생성
            prediction = sess.run(model.prediction, feed_dict=valid_feed_dict)
            # 인덱스 -> 단어 사전을 통해 학습된 결과를 단어로 표현.
            prediction_output = [[reversed_dict[y] for y in x] for x in prediction[:, 0, :]]
            
            # 생성된 예측 결과물을 문장 단위로 저장 
            with open(filename, "w") as f:
                for line in prediction_output:
                    summary = list()
                    for word in line:
                        if word == "</s>":
                            break
                        if word not in summary:
                            summary.append(word)
                    print(" ".join(summary), file=f)
                    
            #with open('./rouge_test/sys_dir/{}.txt'.format(filename), "w") as f:
                #for line in prediction_output:
                    #summary = list()
                    #for word in line:
                        #if word == "</s>":
                            #break
                        #if word not in summary:
                            #summary.append(word)
                    #print(" ".join(summary), file=f)

        print('셰봇이가 5지 선다로 쓸만한 친구들을 추려냈어요.. {}...'.format(filename))

In [5]:
# 학습이 잘된 문장을 선별하여 선지로 만들기 위해 함수 생성
def for_rouge_test(summy, label):
    rouge_dict = {}
    rouge_list = []
    rouge = Rouge()
    
    # 학습 결과로 예측한 결과물을 불러오기
    with open(summy, 'r', encoding = 'utf-8') as f:
            
        for idx, line in enumerate(f.readlines()):
            
            # 실제 문장 - 예측 결과에서 학습의 정도를 Rouge-r과 Rouge-p로 검증
            scores = rouge.get_scores(line, label[idx])
            
            p_score = scores[0]["rouge-1"]["p"] 
            r_score = scores[0]["rouge-1"]["r"] 
            
            # 해당 조건은 재검토가 필요할 듯 
            if r_score > 0: 
                
                # Rouge-r 사용
                r_idx_score = (idx, r_score)            
                
                rouge_list.append(r_idx_score)
                # print(rouge_list)
            
                rouge_dict[idx] = line
            
        # 내림차순으로 가장 점수가 높은 문장을 순서로 리스트 배열
        rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[1], reverse=True)
        # 상위 5개 문장 추출하기
        rouge_list = rouge_list[:5]
        # 오름차순으로 인덱스 순서대로 다시 재배열
        sort_rouge_list = sorted(rouge_list, key = lambda rouge_list: rouge_list[0], reverse=False)
                
        question_list = []
    
        for ix in sort_rouge_list:
            
            question_list.append(rouge_dict[ix[0]])
    # 상위 5개 문장에 대해서 선지로 활용할 수 있도록 리턴      
    return question_list, sort_rouge_list

In [6]:
# 학습 결과물 중 <unk> 부분 처리 
def delete_unk(content, unk_sentence):
    
    container = []
    
    text = open(content, 'r', encoding = 'utf-8').read()
    
    # 각 단어에 대해서 품사 태깅
    tagged_list = pos_tag(word_tokenize(text))
    
    # 명사 부분만 활용
    nnp_list = [t[0] for t in tagged_list if (t[1] == "NNP") or (t[1] == "NN")]
    
    fd_names = FreqDist(nnp_list)
    
    # 빈도수 기반으로 unk 처리하기
    freq = fd_names.most_common(2)
    
    #for tagged in tagged_list:
        
        #if tagged[1] == 'NN' or 'NNP':
            
            #container.append(tagged[0]) # 단어만 추출
    
    
    # unk 부분을 처리하는 방법 중 하나로 최고 빈도수를 기반으로 하는 방식
    replace_sentence = re.sub('< unk >', freq[0][0], unk_sentence)
    # 다중 숫자에 대한 표현은 Several로 대체 
    replace_sentence = re.sub('# nd', 'Several', replace_sentence)
    # 불필요한 줄 바꿈표 제거 
    replace_sentence = re.sub('\n', '', replace_sentence)
    
    return replace_sentence
     


In [7]:
# 워드넷 반의어 사전 활용하기 
def antonym_dict(word):
    syn = []
    ant = []
    for synset in wn.synsets(word):
        for lemma in synset.lemmas():
            syn.append(lemma.name())    #add the synonyms
            if lemma.antonyms():    #When antonyms are available, add them into the list
                ant.append(lemma.antonyms()[0].name())
    # print('Synonyms: ' + str(syn))
    print('Antonyms: ' + str(ant))
    
    #random.shuffle(ant)
    
    if len(ant) > 0:
    
        return ant[0] # string이 최종
    
    else:
        
        return 0 


In [8]:
# 오답 선지 제작하는 함수 
def make_wrong_question(replace_sentence): # 딱 하나만 들어가야 함.
    # 선지로 들어가는 문장을 어절 단위로 나누기
    token = word_tokenize(replace_sentence)
    # 각 어절에 품사 태깅하기 
    tagged_list = pos_tag(token)
    # 형용사, 부사, 동사 일부에 대해서 반의어로 변경
    va_list = [t[0] for t in tagged_list if (t[1] == "JJ") or (t[1] == "JJR") or (t[1] == "JJS")
               or (t[1] == "RB") or (t[1] == "RBS") or (t[1] == "RBR") or (t[1] == "VB") or (t[1] == "VBD")]
    
    
    for va in va_list:
        # 반의어 사전에서 반의어 목록 불러오기
        para = ant_vocab.crawling_dict(va)
        # 반의어가 존재하는지 유무가 결정적
        if len(para) > 0:
            # 있다면? 바꾸어주기 
            token[token.index(va)] = para  
            
    # 리스트 > 문자열로 연결하기
    replace_sentence = ' '.join(token)
    # 최초 문자 대문자로
    replace_sentence = replace_sentence.capitalize() + "."
    
    return replace_sentence
    
    
    
    
    
    
    
    
    
    

In [9]:
# 내용 일치 문제 생성

url = 'https://www.nytimes.com/2019/11/13/health/candida-auris-resistant-hospitals.html'
raw_txt_name = 'CSAT_25.txt'
prepro_txt_name = 'prepro_25.txt'
learned_txt_name = 'correct_25.txt'
num = 1
# 지문의 길이는 뉴스 기사 길이에 따라 유동적으로 설정하기
num_words = 400

def correct_25(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_correct(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_correct(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict) # 5개의 선지가 나옴 

    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == num: # 오답 선지 1개 선택
            # 오답으로 변화시키기
            replace_sentence = make_wrong_question(replace_sentence)
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx != num: # 정답 선지 4개 선택
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사들에 대해서만
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")]
            
            # 유의어 사전을 바탕으로 유사한 의미를 지닌 단어로 변경
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v] # 유의어가 없는 경우가 발생
                
                if paraphrasing_list == list: # 유의어 사전에서 유사어가 있는 경우
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                elif paraphrasing_list != list: # 유사어가 없는 경우 
                    
                    token[token.index(v)] = paraphrasing_list
                    
                    
            
            replace_sentence = ' '.join(token)
                
            replace_sentence = replace_sentence.capitalize() + "."
    
        # 오답 선지 1개, 정답 선지 4개 리스트로 배열
        question_list[idx] = replace_sentence
       
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 넘버링을 포함한 선지로 가공
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('25. 다음 글에 일치하지 않은 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
if __name__ == "__main__":
    
    correct_25(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)
    
    

0 albert was the first patient in the world to receive the antibiotic — penicillin
1 for the past year , matt 's been talking to health experts to find out if we are reaching the end of the antibiotic era
2 is our fate sealed? first off , i don’t think people respect bacteria enough this is ellen silbergeld , one of the leading scientists studying antibiotic resistance
3 no the cdc got our attention today with a warning about what it calls ‘nightmare bacteria’ these are bacteria that are resistant to most , if not all , antibiotics when we take antibiotics to kill infections , some bacteria survive
4 so every time we take an antibiotic , we risk creating stronger , more resistant bacteria
5 and stronger , more resistant bacteria means less and less effective antibiotics
6 drug-resistant bacteria have never been able to travel the world as fast as they do today and that 's just part of the problem
7 you should know that about ## percent of antibiotic production in this country goes into

In [10]:
url = 'https://www.nytimes.com/2019/11/13/sports/baseball/jacob-degrom-cy-young.html'
raw_txt_name = 'CSAT_26.txt'
prepro_txt_name = 'prepro_26.txt'
learned_txt_name = 'correct_26.txt'
num = 1
num_words = 400
def correct_26(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_correct(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_correct(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict) # 5개의 선지가 나옴 
    
    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasin
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx != num: # 오답 선지 4개 
            # 오답으로 변화시키기
            replace_sentence = make_wrong_question(replace_sentence)
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx == num: # 정답 선지 1개
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사들에 대해서만
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")]
            
            # 유의어 사전을 바탕으로 유사한 의미를 지닌 단어로 변경
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v] # 유의어가 없는 경우가 발생
                
                if paraphrasing_list == list: # 유의어 사전에서 유사어가 있는 경우
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                elif paraphrasing_list != list: # 유사어가 없는 경우
                    
                    token[token.index(v)] = paraphrasing_list
                
            
            replace_sentence = ' '.join(token)
                
            replace_sentence = replace_sentence.capitalize() + "."
        
        # 오답 선지 4개, 정답 선지 1개 리스트로 배열
        question_list[idx] = replace_sentence
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 넘버링을 포함한 선지로 가공
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('26. 다음 글에 일치하는 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
if __name__ == "__main__":
    
    correct_26(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)


0 shortly after pitcher jacob degrom signed a contract extension worth $#### million with the mets on the eve of opening day in march , his father , tony , waited for him inside the diplomat room off the lobby at the ritz-carlton in arlington , va
1 tony degrom considered his son 's accomplishments a day before he would start the mets’ opener : rookie of the year in #### , world series runner-up in #### and the #### cy young award winner
2 the elder degrom believed his son 's most recent accomplishment would not be his peak
3 i think he 's capable of winning a few more cy youngs , tony degrom said
4 if you look at his history , he has a lot less wear on him than most guys his age , so i think he 's capable of winning a few more’’
5 the father proved prophetic wednesday night
6 jacob degrom , who battled through a tough stretch in april and overcame what manager mickey callaway called a barking elbow , won the national league cy young award and became the ##th pitcher to claim the prize

In [11]:
# 스포츠 기사 및 연예 기사 활용하여 내용 일치 문제 생성 
url = 'https://www.nytimes.com/2019/11/13/travel/qantas-longest-flight.html'
raw_txt_name = 'CSAT_27.txt'
prepro_txt_name = 'prepro_27.txt'
learned_txt_name = 'correct_27.txt'
num = 1
# 지문의 길이는 뉴스 기사 길이에 따라 유동적으로 설정하기
num_words = 400
def correct_27(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_correct(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_correct(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict) # 5개의 선지가 나옴 

    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == num: # 오답 선지 1개 
            # 오답으로 변화시키기
            replace_sentence = make_wrong_question(replace_sentence)
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx != num: # 정답 선지 4개
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사들에 대해서만
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")]
            
            # 유의어 사전을 바탕으로 유사한 의미를 지닌 단어로 변경
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v] # 유의어가 없는 경우가 발생
                
                if paraphrasing_list == list: # 유의어 사전에서 유사어가 있는 경우
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                elif paraphrasing_list != list: # 유사어가 없는 경우 
                    
                    token[token.index(v)] = paraphrasing_list
                
            replace_sentence = ' '.join(token)
                
            replace_sentence = replace_sentence.capitalize() + "."
        
        # 오답 선지 1개, 정답 선지 4개 리스트로 배열
        question_list[idx] = replace_sentence
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 넘버링을 포함한 선지로 가공
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('27. 다음 글에 일치하지 않은 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
if __name__ == "__main__":
    
    correct_27(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)
    

0 if you’re going to fly for nearly ## hours through multiple time zones dressed in a pair of kangaroo-themed pajamas , jolting in and out of sleep in contravention of your normal circadian rhythm , you should take it easy on the medication
1 no one wants to go crazy in a metal tube ## ,### feet above the pacific
2 on the other hand , suspecting that you have fallen into a rift in the space-time continuum itself is perhaps as reasonable a response as any to the longest (so far) commercial flight in the world
3 so there i was last month , six or so hours into qantas 's first-ever nonstop flight from new york city to sydney
4 it was # am new york time , which made it no o’clock in the tiny upscale refugee camp created by the airline
5 i had been suffering from congestion , the kind that migrates around your sinuses and then becomes an infection in your ear
6 while i was no longer contagious , there was some issue about the future of my ability to hear
7 the internet did not have good new

In [12]:
# 내용 일치 문제 생성

url = 'https://www.nytimes.com/2019/07/04/arts/music/bts-kpop-translators.html?searchResultPosition=1'
raw_txt_name = 'CSAT_28.txt'
prepro_txt_name = 'prepro_28.txt'
learned_txt_name = 'correct_28.txt'
num = 3
# 지문의 길이는 뉴스 기사 길이에 따라 유동적으로 설정하기
num_words = 300

def correct_28(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num):
    
    # 크롤링을 통해 특정 기사 url에서 본문 및 기사 헤드라인을 추출하기 
    text, title = crawl_news(url, raw_txt_name, num_words) # url과 raw_txt_name 파일명 입력
    # 추출한 데이터를 문장별 딕셔너리로 저장하기
    test_dict = bf_create_correct(raw_txt_name, prepro_txt_name) # raw_txt_name은 현재 디렉터리 기준 경로임. 
    # 셰봇이 학습시키기 (자연어 생성, 텍스트 요약)
    shake_bot_correct(learned_txt_name, prepro_txt_name) # 셰봇이 학습시키기 (자연어 요약)
    # Rouge Score에 따라 선지 생성 
    question_list, sort_rouge_list = for_rouge_test(learned_txt_name, test_dict) # 5개의 선지가 나옴 

    # 생성된 선지 (학습의 결과물)을 그대로 사용하지 않고, unk 및 유사어 사전을 활용해서 paraphrasing
    for idx, question in enumerate(question_list):
        
        replace_sentence = delete_unk(prepro_txt_name, question)
        
        if idx == num: # 오답 선지 1개 
            # 오답ㅇ로 변화시키기
            replace_sentence = make_wrong_question(replace_sentence)
            
            replace_sentence = replace_sentence.capitalize() + "."   
    
        if idx != num: # 정답 선지 4개
            
            token = word_tokenize(replace_sentence)
            
            tag = pos_tag(token)
            
            # 동사들에 대해서만
            v_list = [t[0] for t in tag if (t[1] == "VB") or (t[1] == "VBD") or (t[1] == "VBG") or (t[1] == "VBG") or (t[1] == "VBN") or (t[1] == "VBP") or (t[1] == "VBZ")]
            
            # 유의어 사전을 바탕으로 유사한 의미를 지닌 단어로 변경
            for v in v_list:
                
                paraphrasing_dict = sym_vocab.crawling_dict(v)
                
                paraphrasing_list = paraphrasing_dict[v] # 유의어가 없는 경우가 발생
                
                if paraphrasing_list == list: # 유의어 사전에서 유사어가 있는 경우
                
                    random.shuffle(paraphrasing_list)
             
                    token[token.index(v)]= paraphrasing_list[0]
                
                elif paraphrasing_list != list: # 유사어가 없는 경우
                    
                    token[token.index(v)] = paraphrasing_list
                
            
            replace_sentence = ' '.join(token)
                
            replace_sentence = replace_sentence.capitalize() + "."
        
        # 오답 선지 1개, 정답 선지 4개 리스트로 배열
        question_list[idx] = replace_sentence
       
    
    number_list = ['①', '②', '③', '④', '⑤']
    
    # 넘버링을 포함한 선지로 가공
    for ix, final_question in enumerate(question_list):
        
        final_question = number_list[ix] + ' ' + final_question
        
        question_list[ix] = final_question
    
    
    
    """ 실제 문제 만들기 """
    
    print('\n')
    print('\n')
    
    print('28. 다음 글에 일치하지 않는 것을 고르시오.')
    print('-----------------------------------')
    print(text)
    print('\n')
    for question in question_list:
        print(question)
    
if __name__ == "__main__":
    
    correct_28(url, raw_txt_name, prepro_txt_name, learned_txt_name, num_words, num)
    

0 many of these translators got involved because they noticed incorrect or incomplete english transcripts online , but also because they saw an opportunity to participate in the rise of a group they wanted to see succeed
1 at the first bts concert the ##-something fan jiye kim (who posts as @doyou_bangtan) attended , she saw the band wanted the concert to be a way for us to share in our joys and pain , just as humans walking alongside each other , she said in a phone interview 
2 bangtan translations is one of the largest , posting comprehensive , authoritative interpretations of lyrics , tweets and long videos
3 the six-person peachboy team does social media posts , lyrics and letters from the subscription-based fancafe platform
4 for in-depth looks at lyrics , @doolsetbangtan and @doyou_bangtan offer heavily contextualized , almost academic deep dives
5 translating for one of k-pop 's biggest groups comes with pressures
6 the sheer amount of content requires some discernment , even a